# This notebook uses the detect_keyboard_input.ipynb notebook as a template, and adds methods for sending commands to the arduino
- It adds a new global variable, my_board, that stores the serial connection to the Arduino
- It adds six new methods: forward, backward, stop, turn_left, turn_right, and turn_forward that send commands to the arduino
- It adds a setup method, that sets the mode of arduino board pins

In [1]:
# Import packages
import sys
import time

from pymata4 import pymata4
import keyboard  # using module keyboard
import time
import re #regex library

from pymata4 import pymata4 #used to interact with arduino using Firmata protocol

# altered detect_keyboard_input code
- added code to the keyboardInputLoop where the **# your code here comments** are

In [2]:
# which control scheme to use for this robot
def getCommandScheme():
    validInput = False
    while validInput == False:
            userInput = input("Command scheme (0=wasd, 1 = tfgh): ")

            try:
                # ignore all occurences of key presses that could be caused by
                # a command scheme being used.
                # This is a hardcoded list at the moment. It can be changed as needed
                userInput = userInput.replace("w", "")
                userInput = userInput.replace("a", "")
                userInput = userInput.replace("s", "")
                userInput = userInput.replace("d", "")
                userInput = userInput.replace("t", "")
                userInput = userInput.replace("f", "")
                userInput = userInput.replace("g", "")
                userInput = userInput.replace("h", "")

                # convert parsed input to integer
                userInput = int(userInput)
                validInput = True

                #insure that the user input is a valid index in the list
                if userInput < 0 or userInput > 1:
                    validInput = False
                    print("Input out of bounds. Choose a scheme between 0 and 1")
            except:
                print("input must be a number")
                validInput = False


    if userInput == 0:
        return ('w', 'a', 's', 'd')
    else:
        return ('t', 'f', 'g', 'h')
    
def keyboardInputLoop(arduino):
    flagUp = False
    flagLeft = False
    flagDown = False
    flagRight = False
    
    (up, left, down, right) = getCommandScheme()
    
    last = -1
    
    # get user controls and send to robot
    modulo = 3
    counter = 0
    while True:
        if counter == 0:
            try:  # used try so that if user pressed other than the given key error will not be shown
                flagUp = keyboard.is_pressed(up)
                flagLeft = keyboard.is_pressed(left)
                flagDown = keyboard.is_pressed(down)
                flagRight = keyboard.is_pressed(right)
                if keyboard.is_pressed('q'):
                    return
                time.sleep(0.01)
            except:
                if last != 0:
                    print("idle")
                    # Your code here
                    stop(arduino)
                    last = 0
        counter = (counter+1)% modulo
        dY = 0
        dX = 0
        if not (flagUp and flagDown):
            if flagUp:
                dY = 1
            if flagDown:
                dY = -1
        if not (flagLeft and flagRight):
            if flagRight:
                dX = 1
            if flagLeft:
                dX = -1

        if dY == 0 and dX == 0:
            if last != 0:
                print("idle")
                # Your code here
                stop(arduino)
            last = 0
        elif dX == 0:
            if dY == 1:
                if last != 1:
                    print("up")
                    # Your code here
                    forward(arduino)
                last = 1
            if dY == -1:
                if last != 2:
                    print("down")
                    # Your code here
                    backward(arduino)
                last = 2
        elif dY == 0:
            if dX == 1:
                if last != 3:
                    print("right")
                    # Your code here
                    stop(arduino)
                    turn_right(arduino)
                last = 3
            if dX == -1:
                if last != 4:
                    print("left")
                    # Your code here
                    stop(arduino)
                    turn_left(arduino)
                last = 4

# New methods that send commands to the arduino

In [3]:
def forward(board):
    global M1_1, M1_2, M1_PWM, M2_1, M2_2, M2_PWM
    
    board.digital_write(M1_1, 0)
    board.digital_write(M1_2, 1)
    board.pwm_write(M1_PWM, MAX_SPEED)
    
    board.digital_write(M2_1, 0)
    board.digital_write(M2_2, 1)
    board.pwm_write(M2_PWM, MAX_SPEED)

def backward(board):
    global M1_1, M1_2, M1_PWM, M2_1, M2_2, M2_PWM
    
    board.digital_write(M1_1, 1)
    board.digital_write(M1_2, 0)
    board.pwm_write(M1_PWM, MAX_SPEED)
    
    board.digital_write(M2_1, 1)
    board.digital_write(M2_2, 0)
    board.pwm_write(M2_PWM, MAX_SPEED)


def stop(board):
    global M1_1, M1_2, M1_PWM, M2_1, M2_2, M2_PWM
    
    board.pwm_write(M1_PWM, 0)
    board.digital_write(M1_1, 0)
    board.digital_write(M1_2, 0)
    
    board.pwm_write(M2_PWM, 0)
    board.digital_write(M2_1, 0)
    board.digital_write(M2_2, 0)

def turn_left(board):
    global M1_1, M1_2, M1_PWM, M2_1, M2_2, M2_PWM
    
    board.digital_write(M1_1, 1)
    board.digital_write(M1_2, 0)
    board.pwm_write(M1_PWM, MAX_SPEED)
    
    board.digital_write(M2_1, 0)
    board.digital_write(M2_2, 1)
    board.pwm_write(M2_PWM, MAX_SPEED)

def turn_right(board):
    global M1_1, M1_2, M1_PWM, M2_1, M2_2, M2_PWM
    
    board.digital_write(M1_1, 0)
    board.digital_write(M1_2, 1)
    board.pwm_write(M1_PWM, MAX_SPEED)
    
    board.digital_write(M2_1, 1)
    board.digital_write(M2_2, 0)
    board.pwm_write(M2_PWM, MAX_SPEED)
    
    
def setup(arduino):
    # set the pin mode 
    global M1_1, M1_2, M1_PWM, M2_1, M2_2, M2_PWM, M1_reverse, M2_reverse
    
    #swap motor pins if needed
    if M1_reverse:
        M1_1, M1_2 = (M1_2, M1_1)
    if M2_reverse:
        M2_1, M2_2 = (M2_2, M2_1)
    
    #FWD/BCK motor
    arduino.set_pin_mode_digital_output(M1_1)
    arduino.set_pin_mode_digital_output(M1_2)
    arduino.set_pin_mode_pwm_output(M1_PWM)

    #L/R motor
    arduino.set_pin_mode_digital_output(M2_1)
    arduino.set_pin_mode_digital_output(M2_2)
    arduino.set_pin_mode_pwm_output(M2_PWM)
    
    keyboardInputLoop(arduino)


# forward/backward motor pinout
M1_1 = 4            # motor 1 pin 1
M1_2 = 5            # motor 1 pin 2
M1_PWM = 6          # motor 1 PWM pin
M1_reverse = False  # change value if motor is spinning in wrong direction 

# L/R motor pinout
M2_1 = 7            # motor 2 pin 1
M2_2 = 8            # motor 2 pin 2
M2_PWM = 9          # motor 2 PWM pin    
M2_reverse = True  # change value if motor is spinning in wrong direction

MAX_SPEED = 200

# Connect to Arduino board, specifying the COM port
my_board = pymata4.Pymata4("COM6")

pymata4:  Version 1.11

Copyright (c) 2020 Alan Yorinks All Rights Reserved.

Opening COM6...

Waiting 4 seconds(arduino_wait) for Arduino devices to reset...
Arduino compatible device found and connected to COM6

Retrieving Arduino Firmware ID...
Arduino Firmware ID: 1.2 FirmataExpress_editableBaudRate.ino

Retrieving analog map...
Auto-discovery complete. Found 20 Digital Pins and 6 Analog Pins




# Entry point into code

In [4]:
try:
    setup(my_board)
except KeyboardInterrupt:
    board.shutdown()
    sys.exit(0)

Command scheme (0=wasd, 1 = tfgh):  0


idle
up
idle
up
idle
up
idle
down
idle
up
idle
down
idle
left
idle
left
idle
right
idle
right
idle
